### _Activity feature extractor_
Author : @leopauly | cnlp@leeds.ac.uk <br>
Description : For extracting action features from a video

In [1]:
## Imports
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, MaxPooling2D, Input, Concatenate
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
import numpy as np
import pylab as plt
import keras

## Custom scripts
import lscript as lsp
import dataset as dset

Using TensorFlow backend.


In [2]:
## Defining variables
time_step=50 # Number of frames to be skipped
imagefolderpath= ('/nobackup/leopauly/KTH_new/') # Folder pathe where the video sequences are stored
print(os.path.isdir(imagefolderpath)) # Checking if directory of video dataset exists
LOG_DIR='./logdir'
custom_global_step=0
hidden_num = 24 # Number of neurons in hidden layer of LSTM
nb_classes=3
lr_rate=.0005
batch_size = 20
nb_seq_train=360
no_of_batches = int(nb_seq_train / batch_size)
epoch = 5
height=150
width=300
channel=1

True


#### Loading and displaying data

In [ ]:
# TODO: Make display like an animation
x,y=dset.batch_gen_train(nb_classes,batch_size,time_step,height,width,channel,imagefolderpath,gray=True)
item_num=19
lsp.view_video_seq(x,y,time_step,item_num)

#### Defining model

In [ ]:
## Defining placeholders in tf
data_1 = tf.placeholder(tf.float32, [None, time_step, height, width, channel]) #step_size=No: of frames in video sequence
data_2 = tf.placeholder(tf.float32, [None, time_step, height, width, channel])
data_3 = tf.placeholder(tf.float32, [None, time_step, height, width, channel])
target = tf.placeholder(tf.float32, [None, nb_classes])

In [ ]:
## [1] Defining the model in keras using functional layers
input_1 = Input(shape=(None, height, width, channel))
input_2 = Input(shape=(None, height, width, channel))
input_3 = Input(shape=(None, height, width, channel))

conv3D=Conv3D(filters=1, kernel_size=(3,3,3),padding='same')
layer1_1=conv3D(inputs=input_1)
layer1_2=conv3D(inputs=input_1)
layer1_3=conv3D(inputs=input_1)

concatenated = keras.layers.concatenate([layer1_1, layer1_2,layer1_3])
model_out=ConvLSTM2D(filters=1, kernel_size=(5, 5),padding='valid', return_sequences=True)(concatenated)

model=Model([input_1,input_2,input_3],model_out)
inter_out=model([data_1,data_2,data_3])
print(model.summary())

layer2=tf.nn.max_pool3d(inter_out,ksize=(1,1,4,4,1),strides=(1,1,4,4,1),padding='VALID')
size1=tf.size(layer2)
shape1=tf.shape(layer2)
fc = tf.reshape(layer2,[batch_size, 133200]) #shape1[1]*shape1[2]*shape1[3]*shape1[4]])  #138750
#shape2=tf.shape(fc)
out=tf.layers.dense(fc,nb_classes)

In [ ]:
init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)

x,y=dset.batch_gen_train(nb_classes,batch_size,time_step,height,width,channel,imagefolderpath,gray=True)
x =x.reshape(x.shape[0],x.shape[1],height,width,channel)
y_onehot = lsp.one_hot(y,nb_classes)
print(sess.run([out],{data_1: x,data_2:x,data_2:x, target: y_onehot}))

In [ ]:
predictions= tf.nn.softmax(out)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target,logits=out))
optimizer = tf.train.AdamOptimizer(lr_rate)
minimize = optimizer.minimize(cross_entropy)
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(predictions, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

# Defining variables for writing summary
tf.summary.histogram("predictions",predictions )
tf.summary.scalar("cross_entropy",cross_entropy )
summary_op = tf.summary.merge_all()

#### Training model

In [ ]:
#TODO : make data generater to generate class also
init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)

saver = tf.train.Saver()
train_writer = tf.summary.FileWriter('./logdir', sess.graph)

for i in range(epoch):
    ptr = 0
    for iteration in range(no_of_batches):
        #x=dset.batch_gen(0+(batch_size*iteration),batch_size+(batch_size*iteration),batch_size,time_step,height,width,channel,imagefolderpath,gray=True)
        x,y=dset.batch_gen_train(nb_classes,batch_size,time_step,height,width,channel,imagefolderpath,gray=True)
        x =x.reshape(x.shape[0],x.shape[1],height,width,channel)
        y_onehot = lsp.one_hot(y,nb_classes)
        #print(x.shape,y.shape)
        
        summ_string,_=sess.run([summary_op,minimize],{data: x, target: y_onehot})
        train_writer.add_summary(summ_string,custom_global_step+1) # Writing summary to disc
        custom_global_step+=1
        
        print('Iteration {}'.format(iteration))
    print ('Epoch {}'.format(str(i)))

saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))

#### Visualising layers

In [ ]:
x,y=dset.batch_gen_test(nb_classes,batch_size,time_step,height,width,channel,imagefolderpath,gray=True)
x =x.reshape(x.shape[0],x.shape[1],height,width,channel)
y_onehot = lsp.one_hot(y,nb_classes)

img=x[12]
print(img.shape)
img=img.reshape(1,img.shape[0],height,width,channel)
print(img.shape)
y_=np.zeros([1,nb_classes])
units = sess.run(inter_out,feed_dict={data:img, target:y_})
print(units.shape)
units=units.reshape(units.shape[1],units.shape[2])
print(units.shape)
plt.imshow(units)
plt.gray()
plt.savefig('./results/result6.png')
plt.show()

In [ ]:
sess=tf.InteractiveSession() 
saved_model = tf.train.import_meta_graph('./logdir/model.ckpt.meta')
saved_model.restore(sess, tf.train.latest_checkpoint('./logdir'))

In [ ]:
img=x[1]
print(img.shape)
img=img.reshape(1,img.shape[0],height,width,channel)
print(img.shape)
y_=np.zeros([1,nb_classes])
units = sess.run(inter_out,feed_dict={data:img, target:y_})
print(units.shape)
units=units.reshape(units.shape[1],units.shape[2])
print(units.shape)
plt.imshow(units)
plt.gray()
plt.savefig('./results/result3.png')
plt.show()

#### Tensorboard

In [ ]:
# Checking the items in the log directory
!tensorboard --inspect --logdir='./logdir/'

# Running tensorflow
print('If on windows system go to: http://localhost:6006')
!tensorboard --logdir='./logdir/'

In [ ]:
sess.close()